In [ ]:
# pip install -U sentence-transformers

In [ ]:
!pip install corus
! pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c2a61017e92de6e1f49fefb077822a7b4f913924024a265ea151454d95ea8641
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pandas as pd
from corus import load_lenta2
from tqdm import tqdm
from numpy.linalg import norm
import numpy as np

import re
import string
import nltk
from num2words import num2words

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
res = pd.read_csv('/content/drive/My Drive/ИТМО/глубокое обучение на практите/it_resume.csv')

In [ ]:
job = pd.read_excel('/content/drive/My Drive/ИТМО/глубокое обучение на практите/vacancies.xlsx')

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Подготовка датасета с резюме

In [ ]:
res.head(5)

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,index_prog
0,0,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,веб разработчик html css js php базы данных фр...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,3
1,1,"Мужчина , 39 лет , родился 29 июня 1979",40000 руб.,инженер программист,"Старый Оскол , не готов к переезду , готов к р...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 6 месяцев Инженер-программ...,Эльдорадо (Россия),Продавец-консультант,Среднее специальное образование 1996 Профессио...,26.04.2019 05:57,Не указано,12
2,2,"Мужчина , 29 лет , родился 4 декабря 1989",120000 руб.,дизайнер продукта ux ui,"Москва , м. Новогиреево , готов к переезду , ...",полная занятость,полный день,Опыт работы 9 лет 1 месяц Апрель 2016 — по на...,Zenon,"Дизайнер продукта, UX/UI дизайнер",Высшее образование 2012 Московский гуманитарн...,12.04.2019 15:05,Не указано,16
3,3,"Мужчина , 30 лет , родился 6 апреля 1989",260000 руб.,ведущий программист 1c руководитель,"Москва , м. Люблино , готов к переезду , не г...","частичная занятость, полная занятость","полный день, удаленная работа",Опыт работы 9 лет 6 месяцев Ведущий программи...,Сантехника-онлайн.Ру,Ведущий программист 1C,Высшее образование 2012 Московский авиационны...,26.04.2019 11:42,Имеется собственный автомобиль,18
4,4,"Мужчина , 27 лет , родился 8 августа 1991",70000 руб.,программист c web программист разработчик аппа...,"Обнинск , готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день, сменный график, уд...",Опыт работы 5 лет 8 месяцев Март 2019 — по на...,Удаленная работа,Web-программист,Высшее образование 2018 ИАТЭ НИЯУ МИФИ Институ...,17.04.2019 16:49,Имеется собственный автомобиль,21


In [11]:
res.drop(['Unnamed: 0', 'Обновление резюме', 'Авто', 'index_prog'], axis=1, inplace=True)

In [12]:
for col in res.columns:
    print('///////////////////////////')
    print(col)
    print(res[col][0])

///////////////////////////
Пол, возраст
Мужчина ,  38 лет , родился 25 июня 1980
///////////////////////////
ЗП
100000 руб.
///////////////////////////
Ищет работу на должность:
веб разработчик html css js php базы данных фреймворки дизайн интерфейсы cms
///////////////////////////
Город, переезд, командировки
Саратов , не готов к переезду , готов к редким командировкам
///////////////////////////
Занятость
частичная занятость, проектная работа, полная занятость
///////////////////////////
График
гибкий график, удаленная работа
///////////////////////////
Опыт работы
Опыт работы 18 лет 9 месяцев  Август 2017 — Апрель  2019 1 год 9 месяцев OpenSoft Саратов , www.opensoftdev.ru Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Инженер-программист Разработка и поддержка сайтов на корпоративной CMS и на WordPress; разработка корпоративной CMS; дизайн, вёрстка, улучшение интерфейсо

In [ ]:
columns = res.columns
res['all'] = '0'

In [ ]:
columns

Index(['Пол, возраст', 'ЗП', 'Ищет работу на должность:',
       'Город, переезд, командировки', 'Занятость', 'График', 'Опыт работы',
       'Последнее/нынешнее место работы', 'Последняя/нынешняя должность',
       'Образование и ВУЗ'],
      dtype='object')

сделаем колонку, где текстом будут записаны все данные

In [ ]:
for i in tqdm(range(len(res))):
    for col in columns:
        res['all'][i] = res['all'][i] + str(res[col][i])

100%|██████████| 10003/10003 [00:30<00:00, 330.62it/s]


In [ ]:
res['all'][0]

'0Мужчина ,  38 лет , родился 25 июня 1980100000 руб.веб разработчик html css js php базы данных фреймворки дизайн интерфейсы cmsСаратов , не готов к переезду , готов к редким командировкамчастичная занятость, проектная работа, полная занятостьгибкий график, удаленная работаОпыт работы 18 лет 9 месяцев  Август 2017 — Апрель  2019 1 год 9 месяцев OpenSoft Саратов , www.opensoftdev.ru Информационные технологии, системная интеграция, интернет ... Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг Инженер-программист Разработка и поддержка сайтов на корпоративной CMS и на WordPress; разработка корпоративной CMS; дизайн, вёрстка, улучшение интерфейсов существующих сайтов, основанных на разных фреймворках и CMS; консультации и техническая поддержка.  Октябрь 2006 — Июнь  2018 11 лет 9 месяцев СГУ им. Н.Г. Чернышевского Саратовская область , www.sgu.ru/ Преподаватель, научный руководитель Преподаватель (растровая компьютерная графика, разработка 

дополнительно почистим данные

In [ ]:
def text_without_url(text):
    return re.sub(r'http\S+', '', text)

def text_lowercase(text):
    return text.lower()

def remove_punctuation(text):
    res = re.sub(r'[^\w\s]', ' ', text)
    return res

def remove_whitespace(text):
    return  " ".join(text.split())

rus_stop_words = set(stopwords.words('russian'))

In [ ]:
res['all'] = res['all'].apply(lambda x: text_without_url(x))
res['all'] = res['all'].apply(lambda x: text_lowercase(x))
res['all'] = res['all'].apply(lambda x: remove_punctuation(x))
res['all'] = res['all'].apply(lambda x: remove_whitespace(x))

In [ ]:
res['all'][0]

'0мужчина 38 лет родился 25 июня 1980100000 руб веб разработчик html css js php базы данных фреймворки дизайн интерфейсы cmsсаратов не готов к переезду готов к редким командировкамчастичная занятость проектная работа полная занятостьгибкий график удаленная работаопыт работы 18 лет 9 месяцев август 2017 апрель 2019 1 год 9 месяцев opensoft саратов www opensoftdev ru информационные технологии системная интеграция интернет системная интеграция автоматизации технологических и бизнес процессов предприятия ит консалтинг инженер программист разработка и поддержка сайтов на корпоративной cms и на wordpress разработка корпоративной cms дизайн вёрстка улучшение интерфейсов существующих сайтов основанных на разных фреймворках и cms консультации и техническая поддержка октябрь 2006 июнь 2018 11 лет 9 месяцев сгу им н г чернышевского саратовская область www sgu ru преподаватель научный руководитель преподаватель растровая компьютерная графика разработка интерфейсов php javascript фотография и др ди

In [ ]:
res['embedigs'] = ''

In [ ]:
for i in tqdm(range(len(res))):
    sentence = res['all'][i]
    embedding = model.encode(sentence)
    res['embedigs'][i] = embedding

100%|██████████| 10003/10003 [20:53<00:00,  7.98it/s]


In [ ]:
res.to_csv('/content/drive/My Drive/ИТМО/глубокое обучение на практите/it_resume_embedings.csv')

# Подготовка датасета с вакансиями

In [13]:
job.head()

,vac_title_tag,vacancy_title,required_experience,description,skills,address,mode,link
0,"<h1 class=""bloko-header-section-1"" data-qa=""va...",Full-stack Developer (PHP/JS),3–6 лет,топ-3 Мы — в тройке лучших работодателей сегм...,NaN,Калининград,"Полная занятость, полный день",https://adsrv.hh.ru/click?b=730457&c=7&place=3...
1,"<h1 class=""bloko-header-section-1"" data-qa=""va...",Инженер-программист,3–6 лет,"Обязанности: Поддержка, модернизация существую...","C/C++, Разработка ПО","Екатеринбург, Вишнёвая улица, 7","Полная занятость, полный день",https://adsrv.hh.ru/click?b=712735&c=5&place=3...
2,"<h1 class=""bloko-header-section-1"" data-qa=""va...",PL/SQL Developer,1–3 года,"Мы, VADAROD – IT компания, которая специализир...","PL+, PL/SQL, SQL, ORACLE, Java SE, Java EE, Ан...","Минск, Купаловская, Октябрьская, Площадь Побед...","Полная занятость, полный день",https://spb.hh.ru/vacancy/89305950?from=vacanc...
3,"<h1 class=""bloko-header-section-1"" data-qa=""va...",Программист 1C,1–3 года,Что требуется делать: Доработка и разработка к...,NaN,"Бишкек, улица Жусупа Абдрахманова, 170/2","Полная занятость, полный день",https://spb.hh.ru/vacancy/89412541?from=vacanc...
4,"<h1 class=""bloko-header-section-1"" data-qa=""va...",Руководитель направления (конструктор/разработ...,1–3 года,"Приглашаем на работу в крупную, Федеральную ко...",NaN,"Вологда, Новгородская улица, 2А","Полная занятость, полный день",https://spb.hh.ru/vacancy/89611775?from=vacanc...


In [ ]:
job.drop(['vac_title_tag', 'link'], axis=1, inplace=True)

In [ ]:
columns = job.columns
job['all'] = '0'
job['embedigs'] = ''

In [ ]:
job.head()

,vacancy_title,required_experience,description,skills,address,mode,all,embedigs
0,Full-stack Developer (PHP/JS),3–6 лет,топ-3 Мы — в тройке лучших работодателей сегм...,NaN,Калининград,"Полная занятость, полный день",0,
1,Инженер-программист,3–6 лет,"Обязанности: Поддержка, модернизация существую...","C/C++, Разработка ПО","Екатеринбург, Вишнёвая улица, 7","Полная занятость, полный день",0,
2,PL/SQL Developer,1–3 года,"Мы, VADAROD – IT компания, которая специализир...","PL+, PL/SQL, SQL, ORACLE, Java SE, Java EE, Ан...","Минск, Купаловская, Октябрьская, Площадь Побед...","Полная занятость, полный день",0,
3,Программист 1C,1–3 года,Что требуется делать: Доработка и разработка к...,NaN,"Бишкек, улица Жусупа Абдрахманова, 170/2","Полная занятость, полный день",0,
4,Руководитель направления (конструктор/разработ...,1–3 года,"Приглашаем на работу в крупную, Федеральную ко...",NaN,"Вологда, Новгородская улица, 2А","Полная занятость, полный день",0,


сделаем колонку, где текстом будут записаны все данные

In [ ]:
for i in tqdm(range(len(job))):
    for col in columns:
        job['all'][i] = job['all'][i] + str(job[col][i])

100%|██████████| 4746/4746 [00:09<00:00, 477.92it/s]


дополнительная чистка

In [ ]:
job['all'] = job['all'].apply(lambda x: text_without_url(x))
job['all'] = job['all'].apply(lambda x: text_lowercase(x))
job['all'] = job['all'].apply(lambda x: remove_punctuation(x))
job['all'] = job['all'].apply(lambda x: remove_whitespace(x))

In [ ]:
job['all'][0]

'0full stack developer php js 3 6 лет топ 3 мы в тройке лучших работодателей сегмента it и интернет по версии headhunter 2022 средние компании 20 лет мы создаем продукты для управления бизнесом по всему миру 15 место в рейтинге самых быстрорастущих технологических компаний 2020 г 11 место среди компаний рунета в рейтинге forbes 2020 г 10 млн компаний зарегистрировано в битрикс24 по всему миру следующее достижение твоё стань частью команды битрикс24 мы ищем сильных командных игроков с глубокими экспертными знаниями и опытом для создания развития и поддержки сервиса видеоконференций в рамках масштабного международного saas проекта чем тебе предстоит заниматься проектирование архитектуры приложений и сервисов структуры баз и других хранилищ данных написание кода и ревью профилирование и отладка приложений и сервисов для исправления ошибок поиска узких мест и оптимизации участие в улучшении и оптимизации процессов в разработке и иной командной работе включая участие в ревью проектов решени

In [ ]:
for i in tqdm(range(len(job))):
    sentence = job['all'][i]
    embedding = model.encode(sentence)
    job['embedigs'][i] = embedding

100%|██████████| 4746/4746 [08:25<00:00,  9.39it/s]


In [ ]:
job.to_csv('/content/drive/My Drive/ИТМО/глубокое обучение на практите/job_embedings.csv')

# Ранжирование

ищем топ 3 релевантных вакансий по косинусному расстоянию

In [ ]:
def cos(a, b):
    cos_angle = np.dot(a, b) / norm(a) / norm(b)
    return cos_angle

In [ ]:
res['job'] = ''
res.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,all,embedigs,job
0,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,веб разработчик html css js php базы данных фр...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,0мужчина 38 лет родился 25 июня 1980100000 руб...,"[-0.24152121, 0.45695886, -0.23113595, -0.2643...",
1,"Мужчина , 39 лет , родился 29 июня 1979",40000 руб.,инженер программист,"Старый Оскол , не готов к переезду , готов к р...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 6 месяцев Инженер-программ...,Эльдорадо (Россия),Продавец-консультант,Среднее специальное образование 1996 Профессио...,0мужчина 39 лет родился 29 июня 197940000 руб ...,"[-0.09654911, 0.6202426, -0.15133971, -0.03374...",
2,"Мужчина , 29 лет , родился 4 декабря 1989",120000 руб.,дизайнер продукта ux ui,"Москва , м. Новогиреево , готов к переезду , ...",полная занятость,полный день,Опыт работы 9 лет 1 месяц Апрель 2016 — по на...,Zenon,"Дизайнер продукта, UX/UI дизайнер",Высшее образование 2012 Московский гуманитарн...,0мужчина 29 лет родился 4 декабря 1989120000 р...,"[-0.101813406, 0.65865314, -0.3394365, -0.0391...",
3,"Мужчина , 30 лет , родился 6 апреля 1989",260000 руб.,ведущий программист 1c руководитель,"Москва , м. Люблино , готов к переезду , не г...","частичная занятость, полная занятость","полный день, удаленная работа",Опыт работы 9 лет 6 месяцев Ведущий программи...,Сантехника-онлайн.Ру,Ведущий программист 1C,Высшее образование 2012 Московский авиационны...,0мужчина 30 лет родился 6 апреля 1989260000 ру...,"[-0.08650114, 0.50779784, -0.26907197, 0.05320...",
4,"Мужчина , 27 лет , родился 8 августа 1991",70000 руб.,программист c web программист разработчик аппа...,"Обнинск , готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день, сменный график, уд...",Опыт работы 5 лет 8 месяцев Март 2019 — по на...,Удаленная работа,Web-программист,Высшее образование 2018 ИАТЭ НИЯУ МИФИ Институ...,0мужчина 27 лет родился 8 августа 199170000 ру...,"[-0.30889618, 0.48295566, -0.18641312, -0.2055...",


In [ ]:
res['job2'] = ''
res['job3'] = ''

In [ ]:
top1 = [0.000003, -1]
top2 = [0.000002, -1]
top3 = [0.000001, -1]

for j in range(20):
    for i in tqdm(range(len(job))):
        # print(j, i)
        a = res['embedigs'][j]
        b = job['embedigs'][i]
        cosinus = cos(a, b)

        if cosinus > top1[0]:
            top3[0] = top2[0]
            top3[1] = top2[1]

            top2[0] = top1[0]
            top2[1] = top1[1]

            top1[0] =  cosinus
            top1[1] =  i

        elif (cosinus > top2[0]) & (cosinus < top1[0]):
            top3[0] = top2[0]
            top3[1] = top2[1]

            top2[0] = cosinus
            top2[1] = i

        elif (cosinus > top3[0]) & (cosinus < top2[0]):
            top3[0] = cosinus
            top3[1] = i

    res['job'][j] = top1
    res['job2'][j] = top2
    res['job3'][j] = top3

    top1 = [0.000003, -1]
    top2 = [0.000002, -1]
    top3 = [0.000001, -1]

100%|██████████| 4746/4746 [00:00<00:00, 39932.61it/s]


In [ ]:
res.head(20)

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,all,embedigs,job,job2,job3
0,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,веб разработчик html css js php базы данных фр...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,0мужчина 38 лет родился 25 июня 1980100000 руб...,"[-0.24152121, 0.45695886, -0.23113595, -0.2643...","[0.9373709, 530]","[0.9230975, 346]","[0.9035956, 2947]"
1,"Мужчина , 39 лет , родился 29 июня 1979",40000 руб.,инженер программист,"Старый Оскол , не готов к переезду , готов к р...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 6 месяцев Инженер-программ...,Эльдорадо (Россия),Продавец-консультант,Среднее специальное образование 1996 Профессио...,0мужчина 39 лет родился 29 июня 197940000 руб ...,"[-0.09654911, 0.6202426, -0.15133971, -0.03374...","[0.97593004, 1946]","[0.97540194, 3219]","[0.974321, 162]"
2,"Мужчина , 29 лет , родился 4 декабря 1989",120000 руб.,дизайнер продукта ux ui,"Москва , м. Новогиреево , готов к переезду , ...",полная занятость,полный день,Опыт работы 9 лет 1 месяц Апрель 2016 — по на...,Zenon,"Дизайнер продукта, UX/UI дизайнер",Высшее образование 2012 Московский гуманитарн...,0мужчина 29 лет родился 4 декабря 1989120000 р...,"[-0.101813406, 0.65865314, -0.3394365, -0.0391...","[0.96734834, 1376]","[0.9629157, 4614]","[0.9606745, 69]"
3,"Мужчина , 30 лет , родился 6 апреля 1989",260000 руб.,ведущий программист 1c руководитель,"Москва , м. Люблино , готов к переезду , не г...","частичная занятость, полная занятость","полный день, удаленная работа",Опыт работы 9 лет 6 месяцев Ведущий программи...,Сантехника-онлайн.Ру,Ведущий программист 1C,Высшее образование 2012 Московский авиационны...,0мужчина 30 лет родился 6 апреля 1989260000 ру...,"[-0.08650114, 0.50779784, -0.26907197, 0.05320...","[0.96387184, 3492]","[0.9632574, 162]","[0.9624968, 3794]"
4,"Мужчина , 27 лет , родился 8 августа 1991",70000 руб.,программист c web программист разработчик аппа...,"Обнинск , готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день, сменный график, уд...",Опыт работы 5 лет 8 месяцев Март 2019 — по на...,Удаленная работа,Web-программист,Высшее образование 2018 ИАТЭ НИЯУ МИФИ Институ...,0мужчина 27 лет родился 8 августа 199170000 ру...,"[-0.30889618, 0.48295566, -0.18641312, -0.2055...","[0.9627305, 2395]","[0.953417, 2505]","[0.9522335, 2773]"
5,"Мужчина , 33 года , родился 13 июня 1985",60000 руб.,golang разработчик,"Пенза , не готов к переезду , не готов к коман...",полная занятость,"гибкий график, полный день",Опыт работы 11 лет 6 месяцев Golang разработч...,Eastwood,Программист Python,Высшее образование 2007 Пензенский государств...,0мужчина 33 года родился 13 июня 198560000 руб...,"[-0.26616448, 0.5721062, -0.19378515, 0.138782...","[0.95467645, 86]","[0.94784135, 3769]","[0.94665605, 765]"
6,"Мужчина , 34 года , родился 23 июня 1984",61000 руб.,начальник отдела главный специалист аудитор ан...,"Казань , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 12 лет 3 месяца Апрель 2017 — по ...,Банк,главный экономист - аналитик Отдела методологи...,Высшее образование 2013 Университет управления...,0мужчина 34 года родился 23 июня 198461000 руб...,"[-0.09098979, 0.70962644, -0.17697178, 0.05299...","[0.97158027, 3769]","[0.97109824, 1713]","[0.9707146, 191]"
7,"Женщина , 31 год , родилась 18 ноября 1987",3000 бел.руб.,engineer,"Минск , м. Кунцевщина , готова к переезду , г...",полная занятость,полный день,Опыт работы 8 лет 2 месяца Engineer 3 000 бел...,Outsourcing,Data Engineer,Высшее образование 2010 Belorussian State U

In [ ]:
test = res.head(20)
test.to_csv('/content/drive/My Drive/ИТМО/глубокое обучение на практите/test_res.csv')

In [ ]:
top1

[0.9373709, 530]

In [ ]:
top2

[0.9230975, 346]

In [ ]:
job['all'][530]

'0senior php разработчик3 6 летнаша распределенная команда dev насчитывает более 300 специалистов и разрабатывает собственные продукты с 2012 года рекламные платформы игровые сервисы и платформы мобильные игровые и развлекательные приложения у нас проектно ориентированный подход матричная структура с привязкой специалистов к одному проекту но при этом мы работаем командой в работе мы применяем scrum agile приглашаем рнр разработчика от мидла и выше в один из флагманских проектов вас ждет не только поддержка работающего высоконагруженного продукта но и разработка новых фич и сервисов работать можно из любой гео точки график гибкий мы работаем удалённо но для желающих есть офис в москве задачи работа над новыми сервисами продукта решения по оптимизации кода запросов базы данных на проекте решение задач по интеграции разработка и поддержка собственных проектов технологии php 7 symfony сторонние бандлы в т ч sonata admin percona mysql redis rabbitmq mongodb clickhouse мы ждем php опыт рабо

In [ ]:
res['all'][0]

'0мужчина 38 лет родился 25 июня 1980100000 руб веб разработчик html css js php базы данных фреймворки дизайн интерфейсы cmsсаратов не готов к переезду готов к редким командировкамчастичная занятость проектная работа полная занятостьгибкий график удаленная работаопыт работы 18 лет 9 месяцев август 2017 апрель 2019 1 год 9 месяцев opensoft саратов www opensoftdev ru информационные технологии системная интеграция интернет системная интеграция автоматизации технологических и бизнес процессов предприятия ит консалтинг инженер программист разработка и поддержка сайтов на корпоративной cms и на wordpress разработка корпоративной cms дизайн вёрстка улучшение интерфейсов существующих сайтов основанных на разных фреймворках и cms консультации и техническая поддержка октябрь 2006 июнь 2018 11 лет 9 месяцев сгу им н г чернышевского саратовская область www sgu ru преподаватель научный руководитель преподаватель растровая компьютерная графика разработка интерфейсов php javascript фотография и др ди

In [14]:
test = pd.read_csv('/content/drive/My Drive/ИТМО/глубокое обучение на практите/test_res.csv')

In [15]:
test

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,all,embedigs,job,job2,job3
0,0,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,веб разработчик html css js php базы данных фр...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,0мужчина 38 лет родился 25 июня 1980100000 руб...,[-2.41521209e-01 4.56958860e-01 -2.31135949e-...,"[0.9373709, 530]","[0.9230975, 346]","[0.9035956, 2947]"
1,1,"Мужчина , 39 лет , родился 29 июня 1979",40000 руб.,инженер программист,"Старый Оскол , не готов к переезду , готов к р...","частичная занятость, полная занятость","полный день, сменный график",Опыт работы 13 лет 6 месяцев Инженер-программ...,Эльдорадо (Россия),Продавец-консультант,Среднее специальное образование 1996 Профессио...,0мужчина 39 лет родился 29 июня 197940000 руб ...,[-0.09654911 0.6202426 -0.15133971 -0.033743...,"[0.97593004, 1946]","[0.97540194, 3219]","[0.974321, 162]"
2,2,"Мужчина , 29 лет , родился 4 декабря 1989",120000 руб.,дизайнер продукта ux ui,"Москва , м. Новогиреево , готов к переезду , ...",полная занятость,полный день,Опыт работы 9 лет 1 месяц Апрель 2016 — по на...,Zenon,"Дизайнер продукта, UX/UI дизайнер",Высшее образование 2012 Московский гуманитарн...,0мужчина 29 лет родился 4 декабря 1989120000 р...,[-1.01813406e-01 6.58653140e-01 -3.39436501e-...,"[0.96734834, 1376]","[0.9629157, 4614]","[0.9606745, 69]"
3,3,"Мужчина , 30 лет , родился 6 апреля 1989",260000 руб.,ведущий программист 1c руководитель,"Москва , м. Люблино , готов к переезду , не г...","частичная занятость, полная занятость","полный день, удаленная работа",Опыт работы 9 лет 6 месяцев Ведущий программи...,Сантехника-онлайн.Ру,Ведущий программист 1C,Высшее образование 2012 Московский авиационны...,0мужчина 30 лет родился 6 апреля 1989260000 ру...,[-8.65011364e-02 5.07797837e-01 -2.69071966e-...,"[0.96387184, 3492]","[0.9632574, 162]","[0.9624968, 3794]"
4,4,"Мужчина , 27 лет , родился 8 августа 1991",70000 руб.,программист c web программист разработчик аппа...,"Обнинск , готов к переезду , готов к командиро...","частичная занятость, полная занятость","гибкий график, полный день, сменный график, уд...",Опыт работы 5 лет 8 месяцев Март 2019 — по на...,Удаленная работа,Web-программист,Высшее образование 2018 ИАТЭ НИЯУ МИФИ Институ...,0мужчина 27 лет родился 8 августа 199170000 ру...,[-3.08896184e-01 4.82955664e-01 -1.86413124e-...,"[0.9627305, 2395]","[0.953417, 2505]","[0.9522335, 2773]"
5,5,"Мужчина , 33 года , родился 13 июня 1985",60000 руб.,golang разработчик,"Пенза , не готов к переезду , не готов к коман...",полная занятость,"гибкий график, полный день",Опыт работы 11 лет 6 месяцев Golang разработч...,Eastwood,Программист Python,Высшее образование 2007 Пензенский государств...,0мужчина 33 года родился 13 июня 198560000 руб...,[-2.66164482e-01 5.72106183e-01 -1.93785146e-...,"[0.95467645, 86]","[0.94784135, 3769]","[0.94665605, 765]"
6,6,"Мужчина , 34 года , родился 23 июня 1984",61000 руб.,начальник отдела главный специалист аудитор ан...,"Казань , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 12 лет 3 месяца Апрель 2017 — по ...,Банк,главный экономист - аналитик Отдела методологи...,Высшее образование 2013 Университет управления...,0мужчина 34 года родился 23 июня 198461000 руб...,[-0.09098979 0.70962644 -0.17697178 0.052997...,"[0.97158027, 3769]","[0.97109824, 1713]","[0.9707146, 191]"
7,7,"Женщина , 31 год , родилась 18 ноября 1987",3000 бел.руб.,engineer,"Минск , м. Кунцевщина , готова к переезду , г...",полная занятость,полный день,Опыт работы 8 лет 2 месяца Engineer 3 000 бел...,Outsourcing,Data Engineer,Высшее образование 2010 Belorussian Sta

In [16]:
test['all'][0]

'0мужчина 38 лет родился 25 июня 1980100000 руб веб разработчик html css js php базы данных фреймворки дизайн интерфейсы cmsсаратов не готов к переезду готов к редким командировкамчастичная занятость проектная работа полная занятостьгибкий график удаленная работаопыт работы 18 лет 9 месяцев август 2017 апрель 2019 1 год 9 месяцев opensoft саратов www opensoftdev ru информационные технологии системная интеграция интернет системная интеграция автоматизации технологических и бизнес процессов предприятия ит консалтинг инженер программист разработка и поддержка сайтов на корпоративной cms и на wordpress разработка корпоративной cms дизайн вёрстка улучшение интерфейсов существующих сайтов основанных на разных фреймворках и cms консультации и техническая поддержка октябрь 2006 июнь 2018 11 лет 9 месяцев сгу им н г чернышевского саратовская область www sgu ru преподаватель научный руководитель преподаватель растровая компьютерная графика разработка интерфейсов php javascript фотография и др ди

1. [0.9373709, 530]	- php разработчик
2. [0.9230975, 346]	- Backend developer (python)
3. [0.9035956, 2947] - Developer .NET Core

In [17]:
test['all'][1]

'0мужчина 39 лет родился 29 июня 197940000 руб инженер программистстарый оскол не готов к переезду готов к редким командировкамчастичная занятость полная занятостьполный день сменный графикопыт работы 13 лет 6 месяцев инженер программист 40 000 руб информационные технологии интернет телеком программирование разработка системный администратор инженер занятость частичная занятость полная занятость график работы полный день сменный график опыт работы 13 лет 6 месяцев апрель 2015 по настоящее время 4 года 1 месяц эльдорадо россия старый оскол www eldorado ru розничная торговля интернет магазин розничная сеть электроника бытовая техника продавец консультант продажи цифровой и компьютерной техники наставничество октябрь 2009 по настоящее время 9 лет 7 месяцев ооо центр старый оскол строительство недвижимость эксплуатация проектирование строительство жилищное строительство коммерческих объектов торговые площади офисные здания системный администратор ремонт и настройка компьютерной техники сет

1. [0.97593004, 1946]
2. [0.97540194, 3219]
3. [0.974321, 162]

In [18]:
test['all'][2]

'0мужчина 29 лет родился 4 декабря 1989120000 руб дизайнер продукта ux uiмосква м новогиреево готов к переезду готов к командировкамполная занятостьполный деньопыт работы 9 лет 1 месяц апрель 2016 по настоящее время 3 года 2 месяца zenon москва www zenonline ru дизайнер продукта ux ui дизайнер разработка сайта для b2b площадки зенон рекламные поставки определение и анализ ца проектирование тестирование прототипов плотное взаимодействие с отделом программирования отрисовка ui и анимаций отрисовка адаптивных версий август 2015 по настоящее время 3 года 10 месяцев фриланс проектные работы москва веб дизайнер ux ui дизайнер проектировщик интерфейсов проведение встреч с к клиентами и сбор требований к продукту интервьюирование пользователей продуктов создание и последующая доработка прототипов тестирование юзабилити инструменты axure invision adobe ps июль 2013 по настоящее время 5 лет 11 месяцев фриланс проектные работы москва дизайнер копирайтер разработка дизайна сайтов и e mail рассылок

1. [0.96734834, 1376] - Графический дизайнер для маркетплейсов
2. [0.9629157, 4614] - Ведущий специалист отдела информационной безопасности
3. [0.9606745, 69] - Fullstack Developer/DevOps